### Building an image classification model using very little data  

Based on the tutorial by Francois Chollet @fchollet https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html and the workbook by Guillaume Dominici https://github.com/gggdominici/keras-workshop

This tutorial presents several ways to build an image classifier using keras from just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:  

- training a small network from scratch (as a baseline)  
- using the bottleneck features of a pre-trained network  
- fine-tuning the top layers of a pre-trained network  
  
This will lead us to cover the following Keras features:   
  
- fit_generator for training Keras a model using Python data generators  
- ImageDataGenerator for real-time data augmentation  
- layer freezing and model fine-tuning  
- ...and more.  


## Data

Data can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data  
All you need is the train set  
The recommended folder structure is:  

### Folder structure

```python
data/
    train/
        dogs/ ### 1024 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 1024 pictures
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/ ### 416 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 416 pictures
            cat001.jpg
            cat002.jpg
            ...
```
Note : for this example we only consider 2x1000 training images and 2x400 testing images among the 2x12500 available.

The github repo includes about 1500 images for this model. The original Kaggle dataset is much larger. The purpose of this demo is to show how you can build models with smaller size datasets. You should be able to improve this model by using more data.

In [ ]:
#4/qwES3QDmocA8xCr6NAetXoP1zo1ig_gnsKkPD7wFN9ikfoCxAijQ3pw
#'gdrive/My Drive/colab/data/odontologia.xlsx'
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Data loading

In [ ]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

Using TensorFlow backend.


In [ ]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [ ]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

In [ ]:
# dimensions of our images.
img_width, img_height = 100, 100
path='gdrive/My Drive/Colab Notebooks/how_to_make_an_image_classifier-master'
train_data_dir = os.path.join(path, 'data/train')
validation_data_dir = os.path.join(path, 'data/validation')

In [ ]:
os.path.exists(validation_data_dir)


True

### Imports

In [ ]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 704 images belonging to 2 classes.
Found 352 images belonging to 2 classes.


## Small Conv Net

### Model architecture definition

In [ ]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Training

In [ ]:
nb_epoch = 20
_,(train_c0,train_c1)=np.unique(train_generator.classes, return_counts=True)
nb_train_samples=train_c0+train_c1
print(nb_train_samples)
_,(val_c0,val_c1)=np.unique(validation_generator.classes, return_counts=True)
nb_validation_samples=val_c0+val_c1
print(nb_validation_samples)


704
352


In [ ]:
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

In [ ]:
#model.save_weights('models/basic_cnn_20_epochs.h5')
model.save('models/basic_cnn.h5')
del model

In [ ]:
#model.load_weights('models/basic_cnn_20_epochs.h5')
from keras.models import load_model
model = load_model('models/basic_cnn.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 64)        18496     
__________

If your model successfully runs at one epoch, go back and it for 30 epochs by changing nb_epoch above.  I was able to get to an val_acc of 0.71 at 30 epochs.
A copy of a pretrained network is available in the pretrained folder.

### Evaluating on validation set

Computing loss and accuracy :

In [ ]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.4057888857681643, 0.84375]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_no_dataaugmentation.png)

**After ~10 epochs the neural network reach ~70% accuracy. We can witness overfitting, no progress is made over validation set in the next epochs**

## Data augmentation for improving the model

By applying random transformation to our train set, we artificially enhance our dataset with new unseen images.  
This will hopefully reduce overfitting and allows better generalization capability for our network.

Example of data augmentation applied to a picture:
![Example of data augmentation applied to a picture](pictures/cat_data_augmentation.png)

In [ ]:
train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,        # normalize pixel values to [0,1]
        shear_range=0.2,       # randomly applies shearing transformation
        zoom_range=0.2,        # randomly applies shearing transformation
        horizontal_flip=True)  # randomly flip the images

# same code as before
train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

_,(train_c0,train_c1)=np.unique(train_generator_augmented.classes, return_counts=True)
nb_train_samples=train_c0+train_c1
print(nb_train_samples)

Found 704 images belonging to 2 classes.
704


In [ ]:
model.fit_generator(
        train_generator_augmented,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

In [ ]:
model.save_weights('models/augmented_20_epochs.h5')

In [ ]:
#model.load_weights('models_trained/augmented_30_epochs.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [ ]:
scores=model.evaluate_generator(validation_generator, nb_validation_samples)

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 100 epochs :

![Accuracy evolution](pictures/scores_with_dataaugmentation.png)

**Thanks to data-augmentation, the accuracy on the validation set improved to ~80%**

## Using a pre-trained model

The process of training a convolutionnal neural network can be very time-consuming and require a lot of datas.  

We can go beyond the previous models in terms of performance and efficiency by using a general-purpose, pre-trained image classifier.  This example uses VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories. 

On top of it, we add a small multi-layer perceptron and we train it on our dataset.

### VGG16 + small MLP
![VGG16 + Dense layers Schema](pictures/vgg16_original.png)

#### VGG16 model architecture definition

In [ ]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#### Loading VGG16 weights
This part is a bit complicated because the structure of our model is not exactly the same as the one used when training weights.  
Otherwise, we would use the `model.load_weights()` method.

*Note : the VGG16 weights file (~500MB) is not included in this repository. You can download from here :  
https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3*

In [ ]:
import h5py
f = h5py.File(os.path.join(path,'models/vgg16_weights.h5'))
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]    
    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
    elif layer.__class__.__name__ in ['Conv1D', 'Conv2D', 'Conv3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
    layer.set_weights(weights)

f.close()

### Using the VGG16 model to process samples

In [ ]:
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

_,(train_c0,train_c1)=np.unique(train_generator_bottleneck.classes, return_counts=True)
nb_train_samples=train_c0+train_c1
print(nb_train_samples)
_,(val_c0,val_c1)=np.unique(validation_generator_bottleneck.classes, return_counts=True)
nb_validation_samples=val_c0+val_c1
print(nb_validation_samples)



Found 704 images belonging to 2 classes.
Found 352 images belonging to 2 classes.
704
352


This is a long process, so we save the output of the VGG16 once and for all.  

In [ ]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, nb_train_samples//32)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [ ]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, nb_validation_samples//32)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

Now we can load it...

In [ ]:
train_data = np.load(open(os.path.join(path,'models/bottleneck_features_train.npy'), 'rb'))
train_labels = np.array([0] * (train_c0 ) + [1] * (train_c1))

validation_data = np.load(open(os.path.join(path,'models/bottleneck_features_validation.npy'), 'rb'))
validation_labels = np.array([0] * (val_c0 ) + [1] * (val_c1 ))

In [ ]:
train_data.shape

(704, 3, 3, 512)

And define and train the custom fully connected neural network :

In [ ]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
nb_epoch=40
model_top.fit(train_data, train_labels,
          epochs=nb_epoch, batch_size=32,
          validation_data=(validation_data, validation_labels))

Train on 704 samples, validate on 352 samples
Epoch 1/40
704/704 [==============================] - 0s 226us/step - loss: 0.2772 - acc: 0.8665 - val_loss: 2.7021 - val_acc: 0.7642
Epoch 2/40
704/704 [==============================] - 0s 206us/step - loss: 0.2802 - acc: 0.8707 - val_loss: 2.8506 - val_acc: 0.6080
Epoch 3/40
704/704 [==============================] - 0s 219us/step - loss: 0.2627 - acc: 0.8679 - val_loss: 2.7920 - val_acc: 0.7045
Epoch 4/40
704/704 [==============================] - 0s 214us/step - loss: 0.2627 - acc: 0.8580 - val_loss: 2.8489 - val_acc: 0.6335
Epoch 5/40
704/704 [==============================] - 0s 220us/step - loss: 0.2489 - acc: 0.8793 - val_loss: 2.9264 - val_acc: 0.6051
Epoch 6/40
704/704 [==============================] - 0s 246us/step - loss: 0.2641 - acc: 0.8736 - val_loss: 2.6553 - val_acc: 0.7841
Epoch 7/40
704/704 [==============================] - 0s 213us/step - loss: 0.2496 - acc: 0.8736 - val_loss: 2.8039 - val_acc: 0.7074
Epoch 8/40
704/7

The training process of this small neural network is very fast : ~2s per epoch

In [ ]:
model_top.save_weights('models/bottleneck_40_epochs.h5')

### Bottleneck model evaluation

In [ ]:
#model_top.load_weights('models/with-bottleneck/1000-samples--100-epochs.h5')
#model_top.load_weights('/notebook/Data1/Code/keras-workshop/models/with-bottleneck/1000-samples--100-epochs.h5')

Loss and accuracy :

In [ ]:
model_top.evaluate(validation_data, validation_labels)

352/352 [==============================] - 0s 123us/step


[2.8819328944452782, 0.6732954545454546]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_with_bottleneck.png)

**We reached a 90% accuracy on the validation after ~1m of training (~20 epochs) and 8% of the samples originally available on the Kaggle competition !**

In [ ]:
##Fine-tuning the top layers of a a pre-trained network

Start by instantiating the VGG base and loading its weights.

In [ ]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [ ]:
import h5py
f = h5py.File(os.path.join(path,'models/vgg16_weights.h5'))
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
    elif layer.__class__.__name__ in ['Conv1D', 'Conv2D', 'Conv3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
    layer.set_weights(weights)

f.close()

Build a classifier model to put on top of the convolutional model. For the fine tuning, we start with a fully trained-classifer. We will use the weights from the earlier model. And then we will add this model on top of the convolutional base.

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights(os.path.join(path,'models/bottleneck_40_epochs.h5'))

model_vgg.add(top_model)

For fine turning, we only want to train a few layers.  This line will set the first 25 layers (up to the conv block) to non-trainable.

In [ ]:
for layer in model_vgg.layers[:25]:
    layer.trainable = False
print(len(model_vgg.layers))

32


In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model_vgg.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
# prepare data augmentation configuration  . . . do we need this?
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

_,(train_c0,train_c1)=np.unique(train_generator.classes, return_counts=True)
nb_train_samples=train_c0+train_c1
print(nb_train_samples)
_,(val_c0,val_c1)=np.unique(validation_generator.classes, return_counts=True)
nb_validation_samples=val_c0+val_c1
print(nb_validation_samples)

Found 704 images belonging to 2 classes.
Found 352 images belonging to 2 classes.
704
352


In [ ]:
import keras
print(keras.__version__)

2.2.5


In [ ]:
# fine-tune the model
nb_epoch=20
model_vgg.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=20, validation_data=<keras_pre..., steps_per_epoch=22, validation_steps=352)`
  import sys


Epoch 1/20
22/22 [==============================] - 175s 8s/step - loss: 3.6589 - acc: 0.7557 - val_loss: 2.4910 - val_acc: 0.8438
Epoch 2/20
22/22 [==============================] - 32s 1s/step - loss: 3.5553 - acc: 0.7770 - val_loss: 2.4910 - val_acc: 0.8438
Epoch 3/20
22/22 [==============================] - 32s 1s/step - loss: 3.5553 - acc: 0.7770 - val_loss: 2.4910 - val_acc: 0.8438
Epoch 4/20
22/22 [==============================] - 32s 1s/step - loss: 3.5553 - acc: 0.7770 - val_loss: 2.4910 - val_acc: 0.8438
Epoch 5/20
22/22 [==============================] - 32s 1s/step - loss: 3.5553 - acc: 0.7770 - val_loss: 2.4910 - val_acc: 0.8438
Epoch 6/20
22/22 [==============================] - 32s 1s/step - loss: 3.5553 - acc: 0.7770 - val_loss: 2.4910 - val_acc: 0.8438
Epoch 7/20
22/22 [==============================] - 32s 1s/step - loss: 3.5553 - acc: 0.7770 - val_loss: 2.4910 - val_acc: 0.8438
Epoch 8/20
22/22 [==============================] - 32s 1s/step - loss: 3.5553 - acc: 0.7

In [ ]:
model_vgg.save_weights(os.path.join(path,'models/finetuning_20epochs_vgg.h5'))

In [ ]:
model_vgg.load_weights('models/finetuning_20epochs_vgg.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [ ]:
model_vgg.evaluate_generator(validation_generator, nb_validation_samples)

[2.4909977467561313, 0.84375]

In [ ]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.40578888657248835, 0.84375]

In [ ]:
model_top.evaluate(validation_data, validation_labels)

352/352 [==============================] - 0s 96us/step


[2.8819328944452782, 0.6732954545454546]